In [ ]:
pip install numpy pandas matplotlib tensorflow scikit-learn scipy pywavelets tk fpdf openpyxl

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from scipy.fftpack import fft, fftfreq
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks, lombscargle
import pywt

# Hardcoded file path
file_path = "../cleaned_mastersheet.xlsx"
df = pd.read_excel(file_path, skiprows=1)  # Skip the first row as it is junk

# Convert all columns to numeric, forcing errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with column median to prevent data loss
df.fillna(df.median(), inplace=True)

# Identify all available numeric parameters excluding 'Age (Ma)'
parameters = [col for col in df.columns if col != "Age (Ma)"]
age = df["Age (Ma)"].values

# Create output directory
output_dir = "Periodicity_Analysis"
os.makedirs(output_dir, exist_ok=True)

for param in parameters:
    print(f"Processing {param}...")

    # Create folder for each parameter
    param_folder = os.path.join(output_dir, param.replace(" ", "_"))
    os.makedirs(param_folder, exist_ok=True)

    sphericity = df[param].values.reshape(-1, 1)

    # Normalize data
    scaler = MinMaxScaler()
    sphericity_scaled = scaler.fit_transform(sphericity)

    # FFT Analysis
    n = len(sphericity)
    d_age = np.median(np.diff(age))
    fft_values = fft(sphericity.flatten())
    freqs = fftfreq(n, d=d_age)

    plt.figure(figsize=(10, 5))
    plt.plot(freqs[freqs > 0], np.abs(fft_values[freqs > 0]))
    plt.xlabel("Frequency")
    plt.ylabel("Amplitude")
    plt.title(f"Fourier Transform: Periodicity Detection ({param})")
    plt.grid()
    plt.savefig(os.path.join(param_folder, "FFT.png"))
    plt.close()

    # Autocorrelation Function (ACF)
    autocorr_values = acf(sphericity.flatten(), nlags=50)

    plt.figure(figsize=(10, 5))
    plt.stem(range(len(autocorr_values)), autocorr_values, basefmt=" ")
    plt.xlabel("Lag")
    plt.ylabel("Autocorrelation")
    plt.title(f"Autocorrelation Function ({param})")
    plt.savefig(os.path.join(param_folder, "ACF.png"))
    plt.close()

    # Peak Detection
    peaks, _ = find_peaks(sphericity.flatten(), height=np.percentile(sphericity, 75))

    plt.figure(figsize=(10, 5))
    plt.plot(age, sphericity, label="Original Data")
    plt.scatter(age[peaks], sphericity[peaks], color='red', label="Detected Peaks")
    plt.xlabel("Age (Ma)")
    plt.ylabel(param)
    plt.title(f"Peak Detection in Time Series ({param})")
    plt.legend()
    plt.savefig(os.path.join(param_folder, "Peak_Detection.png"))
    plt.close()

    # Lomb-Scargle Periodogram
    angular_freqs = 2 * np.pi * freqs[freqs > 0]
    try:
        power = lombscargle(age, sphericity.flatten(), angular_freqs)
        plt.figure(figsize=(10, 5))
        plt.plot(freqs[freqs > 0], power)
        plt.xlabel("Frequency")
        plt.ylabel("Power")
        plt.title(f"Lomb-Scargle Periodogram ({param})")
        plt.savefig(os.path.join(param_folder, "Lomb_Scargle.png"))
        plt.close()
    except Exception as e:
        print(f"Lomb-Scargle Periodogram failed for {param}: {e}")

    # Continuous Wavelet Transform (CWT)
    wavename = 'cmor1.0-1.0'
    scales = np.logspace(0.1, 2, num=100)
    try:
        coefficients, frequencies = pywt.cwt(sphericity.flatten(), scales, wavename)
        plt.figure(figsize=(10, 6))
        plt.imshow(np.abs(coefficients), aspect='auto', extent=[age.min(), age.max(), scales.min(), scales.max()], cmap='jet', origin='lower')
        plt.colorbar(label="Magnitude")
        plt.xlabel("Age (Ma)")
        plt.ylabel("Scale")
        plt.title(f"Wavelet Transform: Periodicity Representation ({param})")
        plt.savefig(os.path.join(param_folder, "CWT.png"))
        plt.close()
    except Exception as e:
        print(f"Wavelet Transform failed for {param}: {e}")

    print(f"Analysis completed for {param}. Saved in {param_folder}\n")

print("All parameter analyses completed.")